In [ ]:
# from apls_utils

In [19]:
import gdal
import numpy as np
import subprocess

In [20]:
def convertTo8Bit(rasterImageName, outputRaster,
                  outputPixType='Byte',
                  outputFormat='GTiff',
                  rescale_type='rescale',
                  percentiles=[2, 98]):

    srcRaster = gdal.Open(rasterImageName)
    nbands = srcRaster.RasterCount
    if nbands == 3:
        cmd = ['gdal_translate', '-ot', outputPixType, '-of', outputFormat,
               '-co', '"PHOTOMETRIC=rgb"']
    else:
        cmd = ['gdal_translate', '-ot', outputPixType, '-of', outputFormat]

    for bandId in range(srcRaster.RasterCount):
        bandId = bandId+1
        band = srcRaster.GetRasterBand(bandId)
        if rescale_type == 'rescale':
            bmin = band.GetMinimum()
            bmax = band.GetMaximum()
            # if not exist minimum and maximum values
            if bmin is None or bmax is None:
                (bmin, bmax) = band.ComputeRasterMinMax(1)
            # else, rescale
            band_arr_tmp = band.ReadAsArray()
            bmin = np.percentile(band_arr_tmp.flatten(), percentiles[0])
            bmax = np.percentile(band_arr_tmp.flatten(), percentiles[1])

        else:
            bmin, bmax = 0, 65535

        cmd.append('-scale_{}'.format(bandId))
        cmd.append('{}'.format(bmin))
        cmd.append('{}'.format(bmax))
        cmd.append('{}'.format(0))
        cmd.append('{}'.format(255))

    cmd.append(rasterImageName)
    cmd.append(outputRaster)
    print(cmd)
    subprocess.call(cmd)

    return

In [21]:
r_path = r'E:\UPennOneDrive\OneDrive - PennO365\School\UPenn\Spring_2022\CPLN_680\SN3_roads_sample\SpaceNet_Roads_Sample\AOI_4_Shanghai_Roads_Sample\RGB-PanSharpen\RGB-PanSharpen_AOI_4_Shanghai_img144.tif'

In [22]:
convertTo8Bit(r_path, 'test8bit.tiff')

['gdal_translate', '-ot', 'Byte', '-of', 'GTiff', '-co', '"PHOTOMETRIC=rgb"', '-scale_1', '189.0', '708.0', '0', '255', '-scale_2', '339.0', '942.0', '0', '255', '-scale_3', '282.0', '665.0', '0', '255', 'E:\\UPennOneDrive\\OneDrive - PennO365\\School\\UPenn\\Spring_2022\\CPLN_680\\SN3_roads_sample\\SpaceNet_Roads_Sample\\AOI_4_Shanghai_Roads_Sample\\RGB-PanSharpen\\RGB-PanSharpen_AOI_4_Shanghai_img144.tif', 'test8bit.tiff']
